![Logo](https://www.up.edu.mx/wp-content/themes/panamericana/img/logo-footer.png)

# **Tarea 6: Regresión Lineal Múltiple**

## **Asignatura:** Econometría  
## **Profesora:** Mae. Julieta María Sarralde Vera

---

**Nombre del Alumno:**  
Luis Enrique Noguera Gil

**Fecha de entrega:**  
08 de septiembre de 2024


# Auto reload modules

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import scripts.index as index
from scipy.stats import t
from scipy import stats
import fs

In [4]:
MAIN_DIR = fs.open_fs("../../data/raw/HW06")
XLSX_DIR = MAIN_DIR.getsyspath("Tabla_Tarea_FF.xlsx")

In [42]:
DIR = fs.open_fs(".")
NB_DIR = DIR.getsyspath("0.0-leng-HW04.ipynb")

# Extrae los encabezados y genera el índice
headings = index.extract_headings(NB_DIR)
toc = index.generate_toc(headings)

# Muestra el índice generado
# print(toc)

- [**Tarea 4: Regresión lineal Simple Formas Funcionales**](#**tarea-4:-regresión-lineal-simple-formas-funcionales**)
  - [**Asignatura:** Econometría](#**asignatura:**-econometría)
  - [**Profesora:** Mae. Julieta María Sarralde](#**profesora:**-mae.-julieta-maría-sarralde)
- [Auto reload modules](#auto-reload-modules)
  - [Ejercicio](#ejercicio)
    - [Modelo Elasticidad constante – Tabla 1 (Ahorro – Inversión en EUA)](#modelo-elasticidad-constante-–-tabla-1-(ahorro-–-inversión-en-eua))
      - [Elabore un diagrama de dispersión con las variables inversión y nivel de ahorro en mdd originales y transformadas](#elabore-un-diagrama-de-dispersión-con-las-variables-inversión-y-nivel-de-ahorro-en-mdd-originales-y-transformadas)
      - [Obtenga el intervalo de probabilidad al 95% para el pronostico de la inversión cuando el ahorro es de $200 mdd](#obtenga-el-intervalo-de-probabilidad-al-95%-para-el-pronostico-de-la-inversión-cuando-el-ahorro-es-de-$200-mdd)
      - [Obtenga el intervalo de confianza al 95% para la elasticidad de la inversión por el ahorro y interprete sus resultados](#obtenga-el-intervalo-de-confianza-al-95%-para-la-elasticidad-de-la-inversión-por-el-ahorro-y-interprete-sus-resultados)
    - [Modelo de rendimientos decrecientes – Tabla 2 (Ahorro – Inversión en Francia)](#modelo-de-rendimientos-decrecientes-–-tabla-2-(ahorro-–-inversión-en-francia))
      - [Elabore un diagrama de dispersión con las variables inversión y nivel de ahorro en mde originales y transformadas](#elabore-un-diagrama-de-dispersión-con-las-variables-inversión-y-nivel-de-ahorro-en-mde-originales-y-transformadas)
      - [Pruebe la validez estadística de la siguiente afirmación: “La inversión aumentara en mas de 1 mde ante un incremento del 1% en el nivel de ahorro en Francia.” (Utilice un nivel de significancia del 5%)](#pruebe-la-validez-estadística-de-la-siguiente-afirmación:-“la-inversión-aumentara-en-mas-de-1-mde-ante-un-incremento-del-1%-en-el-nivel-de-ahorro-en-francia.”-(utilice-un-nivel-de-significancia-del-5%))
    - [Modelo de crecimiento exponencial – Tabla 3 (Ahorro – Inversión México)](#modelo-de-crecimiento-exponencial-–-tabla-3-(ahorro-–-inversión-méxico))
      - [Estime el modelo y elabore un diagrama de dispersión con las variables inversión y nivel de ahorro en mdpmxn originales y transformadas](#estime-el-modelo-y-elabore-un-diagrama-de-dispersión-con-las-variables-inversión-y-nivel-de-ahorro-en-mdpmxn-originales-y-transformadas)
    - [Modelo Recíproco – Tabla 4 (Ahorro- Inversión Colombia)](#modelo-recíproco-–-tabla-4-(ahorro--inversión-colombia))
      - [Estime el modelo y elabore un diagrama de dispersión con las variables inversión y nivel de ahorro en mdpcop originales y transformadas](#estime-el-modelo-y-elabore-un-diagrama-de-dispersión-con-las-variables-inversión-y-nivel-de-ahorro-en-mdpcop-originales-y-transformadas)
      - [Pruebe la validez estadística de la siguiente afirmación: “Si el nivel de ahorro es muy grande el nivel de inversión no superara los 60,000 mdcop  en Colombia” (Utilice un nivel de significancia del 5%)](#pruebe-la-validez-estadística-de-la-siguiente-afirmación:-“si-el-nivel-de-ahorro-es-muy-grande-el-nivel-de-inversión-no-superara-los-60,000-mdcop--en-colombia”-(utilice-un-nivel-de-significancia-del-5%))
      - [Si se tiene un ahorro e inversión de:](#si-se-tiene-un-ahorro-e-inversión-de:)

In [6]:
# Librerías

import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings("ignore")
sns.set_style("darkgrid")

# 1. Estimación de Regresiones

Considere los datos de la tabla:

| Y | X₂ | X₃ |
|---|----|----|
| 1 |  1 |  2 |
| 3 |  2 |  1 |
| 8 |  3 | -3 |

Con base en estos datos, estime las siguientes regresiones:

- $ Y_i = \alpha_1 + \alpha_2 X_{2i} + u_{1i} $ (1)
- $ Y_i = \lambda_1 + \lambda_3 X_{3i} + u_{2i} $ (2)
- $ Y_i = \beta_1 + \beta_2 X_{2i} + \beta_3 X_{3i} + u_i $ (3)

Nota: Estime sólo los coeficientes y no los errores estándar.

### a) ¿Es $ \alpha_2 = \beta_2 $? ¿Por qué?

### b) ¿Es $ \lambda_3 = \beta_3 $? ¿Por qué?

¿Qué conclusión importante obtiene de este ejercicio?

In [7]:
df = pd.read_excel(XLSX_DIR , header=0, sheet_name='tabla 1')
df

,t,ahorro,inversión
0,0,13599,87957
1,1,4699,23578
2,2,5473,16345
3,3,6119,6550
4,4,8811,10230
5,5,1142,9127
6,6,143,1675
7,7,138,1110
8,8,85,3351
9,9,108,1140


# 2. La demanda de rosas

En la tabla 7.6 (archivo anexo) se presenta la información trimestral sobre estas variables:

- $ Y = $ cantidad de rosas vendidas, docenas
- $ X_2 = $ precio promedio al mayoreo de las rosas, $/docena
- $ X_3 = $ precio promedio al mayoreo de los claveles, $/docena
- $ X_4 = $ ingreso familiar disponible promedio semanal, $/semana
- $ X_5 = $ variable de tendencia que toma valores de 1, 2, y así sucesivamente, durante el periodo 1971-III a 1975-II en el área metropolitana de Detroit.

Se le pide considerar las siguientes funciones de demanda:

- $ Y_t = \alpha_1 + \alpha_2 X_{2t} + \alpha_3 X_{3t} + \alpha_4 X_{4t} + \alpha_5 X_{5t} + u_t $
- $ \ln Y_t = \beta_1 + \beta_2 \ln X_{2t} + \beta_3 \ln X_{3t} + \beta_4 \ln X_{4t} + \beta_5 X_{5t} + u_t $

### a) Estime los parámetros del modelo lineal e interprete los resultados.

### b) Estime los parámetros del modelo log-lineal e interprete los resultados.

### c) $ \beta_2 $, $ \beta_3 $ y $ \beta_4 $ dan respectivamente las elasticidades de la demanda respecto del precio propio, precio cruzado e ingreso. ¿Cuáles son, a priori, los signos de estas elasticidades? ¿Concuerdan estos resultados con las expectativas a priori?

### d) ¿Cómo calcularía las elasticidades precio propio, precio cruzado e ingreso en el modelo lineal?

### e) Con base en el análisis, ¿cuál modelo, si existe, escogería y por qué?

**TABLA 7.6**  
Demanda trimestral de rosas en el área metropolitana de Detroit, de 1971-III a 1975-II

| Año y trimestre | Y (docenas vendidas) | X₂ ($/docena, rosas) | X₃ ($/docena, claveles) | X₄ ($/semana, ingreso familiar) | X₅ (tendencia) |
|-----------------|---------------------|----------------------|-------------------------|----------------------------------|----------------|
| 1971-III        | 11 484              | 2.26                 | 3.49                    | 158.11                           | 1              |
| 1971-IV         | 9 348               | 2.54                 | 2.85                    | 173.36                           | 2              |
| 1972-I          | 8 429               | 3.07                 | 4.06                    | 165.26                           | 3              |
| 1972-II         | 10 079              | 2.91                 | 3.64                    | 172.92                           | 4              |
| 1972-III        | 9 240               | 2.73                 | 3.64                    | 188.42                           | 5              |
| 1972-IV         | 8 862               | 2.77                 | 3.66                    | 198.62                           | 6              |
| 1973-I          | 6 216               | 3.59                 | 3.76                    | 186.28                           | 7              |
| 1973-II         | 8 263               | 2.60                 | 3.13                    | 180.49                           | 8              |
| 1973-III        | 7 315               | 2.89                 | 3.20                    | 183.33                           | 9              |
| 1973-IV         | 7 911               | 2.94                 | 3.12                    | 183.57                           | 10             |
| 1974-I          | 9 656               | 3.77                 | 3.65                    | 181.87                           | 11             |
| 1974-II         | 10 154              | 3.64                 | 3.60                    | 185.00                           | 12             |
| 1974-III        | 9 350               | 2.94                 | 3.25                    | 184.00                           | 13             |
| 1974-IV         | 8 886               | 2.94                 | 3.12                    | 188.20                           | 14             |
| 1975-I          | 7 718               | 4.24                 | 3.58                    | 175.67                           | 15             |
| 1975-II         | 5 872               | 3.69                 | 3.53                    | 188.00                           | 16             |

\* El autor agradece a Joe Walsh por recopilar estos datos de un mayorista en el área metropolitana de Detroit y por su subsecuente procesamiento.


# 3. Desembolsos del presupuesto de defensa de Estados Unidos, 1962-1981

| Año  | Desembolsos del presupuesto de defensa (Y₁) | PNB (X₁) | Ventas/Asistencias militares de Estados Unidos (X₂) | Ventas de la industria aeroespacial (X₃) | Conflictos 100,000+ (X₄) |
|------|---------------------------------------------|----------|-----------------------------------------------------|------------------------------------------|-------------------------|
| 1962 | 51.1                                        | 560.3    | 0.9                                                 | 16.4                                     | 0                       |
| 1963 | 52.3                                        | 590.5    | 0.9                                                 | 16.6                                     | 0                       |
| 1964 | 53.0                                        | 625.4    | 1.0                                                 | 17.3                                     | 0                       |
| 1965 | 49.6                                        | 684.9    | 0.9                                                 | 17.8                                     | 0                       |
| 1966 | 56.8                                        | 749.9    | 1.1                                                 | 20.6                                     | 0                       |
| 1967 | 72.5                                        | 813.0    | 1.5                                                 | 22.4                                     | 0                       |
| 1968 | 81.2                                        | 931.4    | 1.5                                                 | 24.5                                     | 0                       |
| 1969 | 80.1                                        | 984.6    | 1.6                                                 | 25.0                                     | 0                       |
| 1970 | 77.1                                        | 1,038.9  | 1.7                                                 | 21.1                                     | 0                       |
| 1971 | 77.3                                        | 1,185.9  | 1.8                                                 | 21.5                                     | 0                       |
| 1972 | 78.1                                        | 1,278.4  | 1.8                                                 | 21.8                                     | 0                       |
| 1973 | 78.8                                        | 1,434.2  | 1.6                                                 | 20.8                                     | 0                       |
| 1974 | 87.5                                        | 1,480.4  | 1.6                                                 | 20.1                                     | 0                       |
| 1975 | 91.1                                        | 1,552.6  | 1.6                                                 | 18.3                                     | 0                       |
| 1976 | 97.5                                        | 1,735.2  | 1.5                                                 | 15.6                                     | 0                       |
| 1977 | 105.7                                       | 1,978.4  | 1.6                                                 | 15.2                                     | 0                       |
| 1978 | 114.2                                       | 2,153.2  | 1.7                                                 | 15.7                                     | 0                       |
| 1979 | 125.1                                       | 2,334.8  | 1.8                                                 | 16.7                                     | 0                       |
| 1980 | 138.5                                       | 2,633.1  | 1.9                                                 | 16.6                                     | 0                       |
| 1981 | 162.1                                       | 2,937.7  | 1.9                                                 | 17.0                                     | 0                       |

Para explicar el presupuesto de defensa de Estados Unidos, considere el siguiente modelo:

- $ Y_t = \beta_1 + \beta_2 X_{2t} + \beta_3 X_{3t} + \beta_4 X_{4t} + \beta_5 X_{5t} + u_t $

Donde:

- $ Y_t = $ desembolsos del presupuesto de defensa durante el año t, $ miles de millones
- $ X_2t = $ PNB durante el año t, $ miles de millones
- $ X_3t = $ ventas militares de Estados Unidos/ayuda en el año t, $ miles de millones
- $ X_4t = $ ventas de la industria aeroespacial, $ miles de millones
- $ X_5t = $ conflictos militares que implican a más de 100,000 soldados. Esta variable adquiere el valor de 1 cuando participan 100,000 soldados o más, y es igual a cero cuando el número de soldados no llega a 100,000.

Para probar este modelo, se proporciona la información en la tabla 7.8 (archivo anexo).

### a) Estime los parámetros de este modelo y sus errores estándar, y obtenga $ R^2 $ y $ R^2 $ ajustada.

### b) Comente los resultados, considerando cualquier expectativa a priori que tenga sobre la relación entre $ Y $ y las diversas variables $ X $.

### c) ¿Qué otra(s) variable(s) incluiría en el modelo y por qué?

# 4. Gasto de consumo real, ingreso real, riqueza real y tasas de interés reales de Estados Unidos de 1947 a 2000.

En la tabla 7.12 (archivo anexo) se presenta la información de estas variables:

| Año  | C       | Yd      | Riqueza   | Tasa de Interés |
|------|---------|---------|-----------|-----------------|
| 1947 | 976.4   | 1 035.2 | 5 166.8   | -10.351         |
| 1948 | 998.1   | 1 066.5 | 5 679.0   | -4.720          |
| 1949 | 1 009.7 | 1 062.3 | 5 607.4   | -3.056          |
| 1950 | 1 090.1 | 1 178.1 | 5 795.5   | -0.462          |
| 1951 | 1 129.7 | 1 226.0 | 5 678.9   | 0.507           |
| 1952 | 1 197.2 | 1 294.7 | 6 003.8   | -0.270          |
| 1953 | 1 191.4 | 1 345.4 | 6 112.4   | -0.557          |
| 1954 | 1 227.3 | 1 346.6 | 6 220.3   | -2.766          |
| 1955 | 1 324.4 | 1 454.5 | 6 734.3   | -1.556          |
| 1956 | 1 369.8 | 1 513.7 | 6 788.0   | -0.551          |
| 1957 | 1 421.5 | 1 549.1 | 6 766.0   | 1.018           |
| 1958 | 1 503.1 | 1 622.8 | 7 282.5   | -0.252          |
| 1959 | 1 581.7 | 1 700.7 | 7 905.8   | -0.578          |
| 1960 | 1 616.8 | 1 725.1 | 8 287.0   | -0.769          |
| 1961 | 1 541.2 | 1 728.5 | 8 911.9   | -1.263          |
| 1962 | 1 611.1 | 1 819.6 | 9 871.2   | -1.456          |
| 1963 | 1 696.7 | 1 899.0 | 10 995.1  | -1.016          |
| 1964 | 1 815.5 | 2 027.1 | 11 959.1  | -0.464          |
| 1965 | 1 968.3 | 2 166.6 | 12 472.6  | -0.225          |
| 1966 | 2 079.6 | 2 276.3 | 12 844.4  | -0.877          |
| 1967 | 2 106.1 | 2 341.2 | 12 951.1  | -0.694          |
| 1968 | 2 213.6 | 2 494.7 | 12 914.5  | -0.880          |
| 1969 | 2 275.0 | 2 562.5 | 12 951.4  | -0.468          |
| 1970 | 2 303.7 | 2 651.7 | 12 771.3  | -1.146          |
| 1971 | 2 361.5 | 2 706.5 | 13 011.4  | -0.887          |
| 1972 | 2 483.3 | 2 836.2 | 13 423.6  | -0.860          |
| 1973 | 2 577.3 | 3 004.8 | 13 918.8  | -1.144          |
| 1974 | 2 650.1 | 3 105.2 | 13 599.6  | -1.616          |
| 1975 | 2 775.7 | 3 243.5 | 13 648.4  | -0.986          |
| 1976 | 2 868.9 | 3 343.3 | 13 456.8  | 0.657           |

### a) Con la información de la tabla, estime la función de consumo lineal usando los datos de ingreso, riqueza y tasa de interés. ¿Cuál es la ecuación ajustada?

### b) ¿Qué indican los coeficientes estimados sobre las relaciones entre las variables y el gasto de consumo?
